In [ ]:
import os
import sys
currentdir = os.path.dirname(os.path.realpath('__file__'))
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)
print(currentdir, parentdir)

In [ ]:
import yaml
import requests
from norm import *

In [ ]:
content = requests.get('https://raw.githubusercontent.com/108michael/congress-legislators/master/legislators-historical.yaml').content
source = yaml.load(content)

In [ ]:
df = pd.DataFrame(source)

# test columns
df['dict'] = [{'a': 1}] * len(df)
df['list_obj'] = [['a'] * 2] * len(df)
df['list_dict'] = [[{'a':10, 'b': 'hello'}] * 2] * len(df)
df['list_list'] = [[[10]] * 2] * len(df)
df['list_list_list_dict'] = [[[[{'a':10}],]] * 2] * len(df)

# sample group
df_family = df[~df.family.isna()]
df_family.head()

In [ ]:
def normalize3(dataframe):
    __table_info__ = []
    result = []
    # get all series
    series_list = list(dataframe.iteritems())
    print(f'series_list length:: current: {len(series_list)}')
    
    # iterate through each column/ series
    for name, seri in series_list:
        # type of the object of series' values
        inside = seri.apply(lambda x: type(x)).iloc[0]
        
        # case if it is list -- expand rowwise
        if inside == list:
            print('normalizing:: ', name, inside)
            temp = get_column_list(seri)
            temp = reset_index(temp)
            series_list.extend(list(temp.iteritems()))
        
        # case if it is dict -- expand columnwise
        elif inside == dict:
            print('normalizing:: ', name, inside)
            temp = get_column_dict(seri)
            series_list.extend(list(temp.iteritems()))
        
        # case of all others
        else:
            temp = get_column_obj(seri)
            # __table_info__.append((temp.?, len(temp), ))
            result.append(temp)
    print('exports :: ', len(result))
    return result, pd.DataFrame(__table_info__)

In [ ]:
final, table_info = normalize3(df_family.iloc[:4])

In [ ]:
final[4]

In [ ]:
df_family.family.apply(lambda x: len(x)).sum()

In [ ]:
df_family.name.iloc[0]

In [ ]:
get_list_obj(df_family.terms).head()

In [ ]:
df_family.terms.iloc[0]

In [ ]:
def get_list_dict(series):
    """ returns dataframe of dict object expanded to rows and columns """ 
    ds = series.copy()

    # add index to dict keys
    for ind, rows in ds.iteritems():
        for item in rows:
            item.update({'index': ind})
    
    # normalize ds
    data = ds.apply(lambda x: pd.io.json.json_normalize(x)).values
    df = pd.concat(data, sort=True)
    
    return set_index(df)

In [ ]:
get_list_obj(df_family.family).iloc[0]

In [ ]:
def get_column_list(series):
    """ returns dataframe  """
    inside = series.apply(lambda x: type(x)).iloc[0]
    print(series.name, inside)
    print(series.iloc[0])
    
    if inside == list:
        return get_column_list(get_list_obj(series))
    # elif inside == dict:
    #    return get_list_dict(series)
    else:
        return series
get_column_list(df_family.terms)#.iloc[0]

In [ ]:
get_list_obj(get_column_dict(df_family.id).fec)

In [ ]:
get_list_obj(get_column_dict(df_family.id).fec)

In [ ]:
#len([print(i, v.columns.values) for i, v in enumerate(final)])
final[8]['index']
#final[2]['index']
print([len(i) for i in final])

In [ ]:
reduce(lambda x, y: (pd.merge(x, y)), final[0:28])

In [ ]:
reduce(lambda x, y: (pd.merge(x, y)), final[0:29])['index'].value_counts()

In [ ]:
final[23]

In [ ]:
reduce(lambda x, y: (pd.merge(x, y, how='inner')), final[25:32])

In [ ]:
pd.merge(final[0], final[0])